# Market Sentiment Analysis
This notebook analyzes **real-time financial news sentiment** using:
- **SerpAPI** for fetching the latest financial news.
- **OpenAI GPT-4.0 Mini** for sentiment classification.
- **Pandas** for data processing and report generation.

The goal is to provide **market risk insights** based on real-time data.

# Real-Time Market Sentiment Analysis
This notebook fetches **real-time financial news** using **SerpAPI**, analyzes sentiment with **GPT-4.0 Mini**, and generates a **Market Risk Summary**.

## 1️⃣ Install & Import Dependencies
Ensure all required libraries are installed before proceeding.

In [11]:
!pip install openai langchain langchain-openai

In [12]:
!pip install openai serpapi pandas

## 2️⃣ Fetch Real-Time Financial News
Use **SerpAPI** to retrieve the latest financial headlines. These will be analyzed to assess market sentiment.

In [13]:
!pip uninstall google-search-results -y


Found existing installation: google_search_results 2.4.2
Uninstalling google_search_results-2.4.2:
  Successfully uninstalled google_search_results-2.4.2


## 3️⃣ Analyze Sentiment Using GPT-4
Each headline is passed to **GPT-4.0 Mini**, which classifies the sentiment as **Positive, Negative, or Neutral**.

In [14]:
!pip install google-search-results --upgrade


  Using cached google_search_results-2.4.2-py3-none-any.whl


## 4️⃣ Generate Market Sentiment Report
Summarizes the sentiment distribution and provides insights into overall market risk.

In [15]:
import serpapi
print(dir(serpapi))


['AppleAppStoreSearch', 'BaiduSearch', 'BingSearch', 'DuckDuckGoSearch', 'EbaySearch', 'GoogleScholarSearch', 'GoogleSearch', 'HomeDepotSearch', 'NaverSearch', 'SerpApiClient', 'WalmartSearch', 'YahooSearch', 'YandexSearch', 'YoutubeSearch', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'apple_app_store_search', 'baidu_search', 'bing_search', 'constant', 'duck_duck_go_search', 'ebay_search', 'google_scholar_search', 'google_search', 'home_depot_search', 'naver_search', 'pagination', 'serp_api_client', 'serp_api_client_exception', 'walmart_search', 'yahoo_search', 'yandex_search', 'youtube_search']


## 5️⃣ Conclusion
This approach allows for **automated, real-time sentiment tracking** to assist risk managers in making informed decisions.
Future improvements could include **more advanced sentiment scoring**, integration with **trading algorithms**, or expansion to **specific stock categories**.

In [ ]:
import os
import requests

# Load SerpAPI key from environment
SERPAPI_KEY = os.getenv("SERPAPI_API_KEY")
if not SERPAPI_KEY:
    raise RuntimeError("Missing SERPAPI_API_KEY environment variable.")

# Define search parameters
params = {
    "q": "stock market news",
    "tbm": "nws",  # News search mode
    "api_key": SERPAPI_KEY
}

# Make a request to SerpAPI
response = requests.get("https://serpapi.com/search", params=params)
response.raise_for_status()

# Parse the response
data = response.json()
news_results = data.get("news_results", [])

# Print the first few news headlines
for article in news_results[:5]:
    print(article["title"])


Stock market today: Dow, S&P 500, Nasdaq edge higher amid latest inflation test, Trump's next tariff salvo
Stocks rise as investors try to shake off inflation and tariff concerns: Live updates
Stock Market Today: S&P 500, Nasdaq Rise; Robinhood, AppLovin Stocks in Focus — Live Updates
Stock Market Today: Dow, S&P 500, Nasdaq Open Up After PPI Data; SMCI, Nvidia, Intel, More Movers
Stock Market Today: AppLovin Rockets 38%, Triggers Unique Hold Rule As Indexes Climb (Live Coverage)


In [ ]:
import os
from openai import OpenAI

# Load OpenAI API key from environment
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise RuntimeError("Missing OPENAI_API_KEY environment variable.")

client = OpenAI(api_key=OPENAI_API_KEY)

def analyze_sentiment_gpt4(text):
    """Uses GPT-4.0 Turbo to classify sentiment."""
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": "Analyze the sentiment of the given financial news headline. Reply with only one word: Positive, Negative, or Neutral."},
            {"role": "user", "content": text},
        ],
        temperature=0,
    )
    return response.choices[0].message.content.strip()


In [ ]:
# (Consolidated in previous cell) This cell is intentionally left minimal to avoid duplicating API key usage.
from openai import OpenAI  # noqa: F401


In [ ]:
# Analyze sentiment of each news headli
sentiment_results = {article["title"]: analyze_sentiment_gpt4(article["title"]) for article in news_results[:5]}

# Print sentiment results
for headline, sentiment in sentiment_results.items():
    print(f"Headline: {headline}\nSentiment: {sentiment}\n")



Headline: Stock market today: Dow, S&P 500, Nasdaq edge higher amid latest inflation test, Trump's next tariff salvo
Sentiment: Neutral

Headline: Stocks rise as investors try to shake off inflation and tariff concerns: Live updates
Sentiment: Positive

Headline: Stock Market Today: S&P 500, Nasdaq Rise; Robinhood, AppLovin Stocks in Focus — Live Updates
Sentiment: Positive

Headline: Stock Market Today: Dow, S&P 500, Nasdaq Open Up After PPI Data; SMCI, Nvidia, Intel, More Movers
Sentiment: Positive

Headline: Stock Market Today: AppLovin Rockets 38%, Triggers Unique Hold Rule As Indexes Climb (Live Coverage)
Sentiment: Positive



In [20]:
import pandas as pd

def generate_market_summary(sentiment_results):
    """Creates a market sentiment report based on sentiment distribution."""
    sentiment_counts = pd.Series(sentiment_results.values()).value_counts(normalize=True) * 100
    summary = "📈 **Market Sentiment Report:**\n\n"
    
    for sentiment, percentage in sentiment_counts.items():
        summary += f"{sentiment}: {percentage:.2f}%\n"
    
    if sentiment_counts.get("Negative", 0) > 50:
        summary += "\n⚠️ **High risk detected:** Market sentiment is largely negative. Caution advised."
    elif sentiment_counts.get("Positive", 0) > 50:
        summary += "\n✅ **Market sentiment is largely positive. Growth opportunities ahead.**"
    else:
        summary += "\n🔸 **Market sentiment is neutral. Mixed signals observed.**"

    return summary

# Generate and print summary
market_summary = generate_market_summary(sentiment_results)
print(market_summary)


📈 **Market Sentiment Report:**

Positive: 80.00%
Neutral: 20.00%

✅ **Market sentiment is largely positive. Growth opportunities ahead.**
